In [66]:
# Importar las bibliotecas necesarias
from quantus.metrics.faithfulness.faithfulness_estimate import FaithfulnessEstimate
import quantus

from transformers import AutoModelForSequenceClassification, AutoTokenizer

from utils.utils import load_dataset, save_explanation

from lime.lime_text import LimeTextExplainer

from models.roberta_model import classify_comment, load_roberta_model_main
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [67]:
# Cargar el conjunto de datos
# Ajusta la ruta según sea necesario
dataset = load_dataset(
    "models/relevance_classification/dataset/facebook_labeled.csv", sep=",")
BATCH_SIZE = 8
x_batch_dataset = dataset['Review'][:BATCH_SIZE]
print(x_batch_dataset.to_list())
# Cargar el modelo RoBERTa preentrenado
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# roberta_model = load_roberta_model_main()
roberta_model = AutoModelForSequenceClassification.from_pretrained(
    "modelo de ray mejorado(SHAP)/")
roberta_model.eval()  # Poner el modelo en modo de evaluación

# Crear una instancia de la métrica Estimación de Fidelidad
metric_fidelity = quantus.FaithfulnessEstimate()

['i hate that i cant change my cover photo on the app...\r\n', 'i have to go to the web for that and i cant organize my photos in certain albums only in the web!\r\n', "i also can't change who views my information and stuff on my wall only in web...\r\n", 'this version crashes all the time.\r\n', "it's slow little bit.\r\n", 'sd card storage need\r\n', "it doesn't notify me of close friends updates.\r\n", "i can't post any comments or status updates from my motorola razr\r\n"]


c:\Users\naybv\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [68]:
from explainability_techniques.lime import LIMEExplainer

# Configurar el explicador LIME
explainer_roberta = LIMEExplainer(class_names=['not relevant', 'relevant'])

In [69]:

# Seleccionar una instancia del conjunto de datos para explicar
instance_roberta = dataset['Review'].iloc[0]

In [70]:

# Generar la explicación usando LIME
explanation_roberta = explainer_roberta.explain_instance(
    instance_roberta,
    classify_comment
)

d:\Universidad\informatica 4 año\Semestre 1\Precticas 2\PP2_XAI\models\roberta_model.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(


In [72]:
html_roberta = explanation_roberta.as_html()
save_explanation(html_roberta, 'roberta_explanation.html')
explainer_roberta.save_and_open_html(html_roberta, 'roberta_explanation.html')

In [79]:
import numpy as np
from sympy import false



# Generar explicaciones para todas las instancias del conjunto de datos


saliency_list = []


# Inicializar listas para almacenar las opiniones y etiquetas filtradas


filtered_reviews = []


filtered_labels = []



# for index in range(len(dataset)):


for index in range(x_batch_dataset.size):


    instance_roberta = dataset['Review'].iloc[index]


    # Generar la explicación usando LIME
    explanation_roberta = explainer_roberta.explain_instance(
        instance_roberta,
        classify_comment  # Asegúrate de que esta función esté definida para clasificar el comentario
    )


    # Extraer las atribuciones de saliencia


    # Obtiene las atribuciones de la clase relevante


    saliency_values = explanation_roberta.as_map()[1]
    print(saliency_values, "mapaaa")


    if isinstance(saliency_values, list):


        print(f"Longitud de saliency_values para instancia {


              index}: {len(saliency_values)}")


        # Solo agregar si tiene una longitud específica (por ejemplo, 10)


        if len(saliency_values) == 10:


            saliency_list.append(saliency_values)


            # Agregar la reseña filtrada


            filtered_reviews.append(instance_roberta)


            # Agregar la etiqueta correspondiente


            filtered_labels.append(dataset['Relevant'].iloc[index])


        else:
            # Rellenar con ceros hasta llegar a 10
            padded_saliency = saliency_values + \
                [(-1, 0)] * (10 - len(saliency_values))
            print(padded_saliency, "ceroooooooooo")
            saliency_list.append(padded_saliency)
            # Agregar la reseña filtrada
            filtered_reviews.append(instance_roberta)
            # Agregar la etiqueta correspondiente
            filtered_labels.append(dataset['Relevant'].iloc[index])

print(saliency_list, "saliency_list")
# a_batch_saliency = np.array(saliency_list)
# print(a_batch_saliency.shape, "saliency_list")

# # Preparar los datos para la evaluación
# data_batch = {
#     'x_batch': filtered_reviews,  # Cambiar 'reviews' a 'x_batch'
#     'y_batch': filtered_labels     # Cambiar 'labels' a 'y_batch'
# }
# # Generar lotes a partir del diccionario de datos
# batches = metric_fidelity.generate_batches(data_batch, BATCH_SIZE)
# print(batches, "batchessssssssssssssssssssssssssssssssssssssssss")
# # Evaluar usando general_preprocess y luego calcular las puntuaciones
# for batch in batches:
#     x_batch = np.array(batch['x_batch'])
#     y_batch = np.array(batch['y_batch'])


#     print(x_batch.shape, " ver dimencoonessssssss")
#     # Preprocesar los datos
#     metric_fidelity.general_preprocess(roberta_model, x_batch, y_batch, a_batch_saliency, s_batch=None, channel_first=false,
#                                        explain_func=None, explain_func_kwargs={}, model_predict_kwargs={}, softmax=false, device=None, custom_batch=None)

#     # Aquí puedes realizar la evaluación usando la métrica FaithfulnessEstimate
#     scores = metric_fidelity(roberta_model, x_batch, y_batch)

#     print("Scores:", scores)


# Convertir a arreglos de NumPy


a_batch_saliency = np.array(saliency_list)
print(a_batch_saliency.shape, "a_batch_saliency")
print(a_batch_saliency, "a_batch_saliency")

x_batch = np.array(filtered_reviews)
print(x_batch_dataset.shape, "x_batch_dataset")
print(x_batch_dataset, "x_batch_dataset")

print(x_batch.shape, "x_batch")
print(x_batch, "x_batch")

y_batch = np.array(filtered_labels)
print(y_batch.shape, "y_batch")
print(y_batch, "y_batch")


print(filtered_reviews)


# # Comprobar dimensiones nuevamente


# print("Dimensiones de a_batch_saliency:", a_batch_saliency.shape)



# saliency_list.append(saliency_values)



# # Convertir a arreglos de NumPy y guardar


# a_batch_saliency = np.array(saliency_list)


# print("Dimensiones de a_batch_saliency:", a_batch_saliency.shape)



# Tokenizar los textos
inputs = tokenizer(filtered_reviews, return_tensors='pt',
                   padding=True, truncation=True)
roberta_model.eval()
with torch.no_grad():
    outputs = roberta_model(**inputs)
    logits = outputs.logits  # Acceder a los logits
print(logits.shape)
# Obtener los embeddings de la capa de embedding
with torch.no_grad():
    # Extraer los embeddings
    x_batch_embeddings = roberta_model.roberta.embeddings(inputs['input_ids'])


print(x_batch_embeddings.type)


np.save("explanations_lime_roberta.npy", a_batch_saliency)

d:\Universidad\informatica 4 año\Semestre 1\Precticas 2\PP2_XAI\models\roberta_model.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(
c:\Users\naybv\A

[(4, 0.07903217621254756), (2, 0.057631094684298965), (6, 0.057631094684298965), (10, 0.05112344292713819), (5, 0.05112344292713819), (0, -0.02790873328540935), (9, -0.02790873328540935), (1, -0.02140108152824858), (8, 0.01524578594152915), (3, 0.0065076517571607696)] mapaaa
Longitud de saliency_values para instancia 0: 10


d:\Universidad\informatica 4 año\Semestre 1\Precticas 2\PP2_XAI\models\roberta_model.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(
c:\Users\naybv\A

[(9, 0.056028974225708995), (8, 0.03959431879583892), (3, -0.016434655429870063), (2, 0.014872613015319957), (5, 0.014872613015319957), (7, 0.014872613015319957), (12, 0.014872613015319957), (15, 0.014872613015319957), (0, -0.0015620424145501074), (1, -0.0015620424145501074)] mapaaa
Longitud de saliency_values para instancia 1: 10


d:\Universidad\informatica 4 año\Semestre 1\Precticas 2\PP2_XAI\models\roberta_model.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(
c:\Users\naybv\A

...

[(2, 0.15300128649749434), (3, 0.11108778801920827), (0, 0.07229618513139287), (1, -0.04191349847828613)] mapaaa
Longitud de saliency_values para instancia 5: 4
[(2, 0.15300128649749434), (3, 0.11108778801920827), (0, 0.07229618513139287), (1, -0.04191349847828613), (-1, 0), (-1, 0), (-1, 0), (-1, 0), (-1, 0), (-1, 0)] ceroooooooooo


d:\Universidad\informatica 4 año\Semestre 1\Precticas 2\PP2_XAI\models\roberta_model.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(
c:\Users\naybv\A

[(3, 0.20063688355749343), (1, 0.1849094273712396), (2, 0.13877714220713055), (8, -0.058804561931325126), (7, -0.058804561931325126), (0, -0.046132285164109055), (4, 0.015727456186253822), (6, 0.015727456186253822), (5, 0.0030551794190377753)] mapaaa
Longitud de saliency_values para instancia 6: 9
[(3, 0.20063688355749343), (1, 0.1849094273712396), (2, 0.13877714220713055), (8, -0.058804561931325126), (7, -0.058804561931325126), (0, -0.046132285164109055), (4, 0.015727456186253822), (6, 0.015727456186253822), (5, 0.0030551794190377753), (-1, 0)] ceroooooooooo


d:\Universidad\informatica 4 año\Semestre 1\Precticas 2\PP2_XAI\models\roberta_model.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(
c:\Users\naybv\A

[(3, 0.027255866017297826), (11, 0.027255866017297826), (1, 0.024431482334936507), (2, 0.024431482334936507), (10, 0.024431482334936507), (0, -0.006165582399263378), (6, -0.0028243836823613248), (8, -0.0028243836823613248), (9, -0.0028243836823613248), (12, -0.0028243836823613248)] mapaaa
Longitud de saliency_values para instancia 7: 10
[[(4, 0.07903217621254756), (2, 0.057631094684298965), (6, 0.057631094684298965), (10, 0.05112344292713819), (5, 0.05112344292713819), (0, -0.02790873328540935), (9, -0.02790873328540935), (1, -0.02140108152824858), (8, 0.01524578594152915), (3, 0.0065076517571607696)], [(9, 0.056028974225708995), (8, 0.03959431879583892), (3, -0.016434655429870063), (2, 0.014872613015319957), (5, 0.014872613015319957), (7, 0.014872613015319957), (12, 0.014872613015319957), (15, 0.014872613015319957), (0, -0.0015620424145501074), (1, -0.0015620424145501074)], [(2, 0.04771176691899108), (4, 0.04771176691899108), (6, 0.04771176691899108), (13, 0.04771176691899108), (3, 0.

In [28]:
import quantus

# Listar todas las métricas disponibles en quantus
available_metrics = [attr for attr in dir(quantus) if not attr.startswith('_')]
print(available_metrics)

['AUC', 'AVAILABLE_METRICS', 'AVAILABLE_NORMALISATION_FUNCTIONS', 'AVAILABLE_N_BINS_ALGORITHMS', 'AVAILABLE_PERTURBATION_FUNCTIONS', 'AVAILABLE_SIMILARITY_FUNCTIONS', 'AVAILABLE_XAI_METHODS_CAPTUM', 'AVAILABLE_XAI_METHODS_TF', 'Any', 'AttributionLocalisation', 'AvgSensitivity', 'Callable', 'Completeness', 'Complexity', 'Consistency', 'Continuity', 'DEPRECATED_XAI_METHODS_CAPTUM', 'DEPRECATED_XAI_METHODS_TF', 'Dict', 'EffectiveComplexity', 'EfficientMPRT', 'FaithfulnessCorrelation', 'FaithfulnessEstimate', 'Final', 'Focus', 'IROF', 'Infidelity', 'InputInvariance', 'List', 'LocalLipschitzEstimate', 'MPRT', 'Mapping', 'MaxSensitivity', 'Metric', 'ModelInterface', 'ModelParameterRandomisation', 'Monotonicity', 'MonotonicityCorrelation', 'NonSensitivity', 'Optional', 'PixelFlipping', 'PointingGame', 'PyTorchModel', 'ROAD', 'RandomLogit', 'RegionPerturbation', 'RelativeInputStability', 'RelativeOutputStability', 'RelativeRepresentationStability', 'RelevanceMassAccuracy', 'RelevanceRankAccura

In [82]:
import quantus
from quantus.metrics.faithfulness.faithfulness_estimate import FaithfulnessEstimate



# Dimensiones transformadas: (1407, 10)



# Crear una instancia de la métrica Estimación de Fidelidad


metric_fidelity = quantus.FaithfulnessEstimate()



print("Dimensiones de x_batch:", x_batch.shape)


print("Dimensiones de :", y_batch.shape)


print("Dimensiones de a_batch_saliency:", a_batch_saliency.shape)
print("Dimensiones de embedding:", x_batch_embeddings[:, :, :].shape)

print("a_batch_saliency:", a_batch_saliency)
print("x_batch_embeddings:", x_batch_embeddings)


def pad_arrays_to_max_shape(array1, array2):
    # Obtener las formas de ambos arrays
    shape1 = array1.shape
    shape2 = array2.shape

    # Determinar la nueva forma
    new_shape = []

    for dim1, dim2 in zip(shape1, shape2):
        new_shape.append(max(dim1, dim2))

    # Rellenar array1 si es necesario
    if shape1 != tuple(new_shape):
        padding_width = [(0, new_shape[i] - shape1[i])
                         for i in range(len(shape1))]
        array1 = np.pad(array1, padding_width,
                        mode='constant', constant_values=0)

    # Rellenar array2 si es necesario
    if shape2 != tuple(new_shape):
        padding_width = [(0, new_shape[i] - shape2[i])
                         for i in range(len(shape2))]
        array2 = np.pad(array2, padding_width,
                        mode='constant', constant_values=0)

    return array1, array2


# Llamar a la función para ajustar las dimensiones
a_batch_saliency_padded, x_batch_embeddings_padded = pad_arrays_to_max_shape(
    a_batch_saliency[:, :, 0], logits)
# Verificar las nuevas dimensiones
print("Dimensiones a_batch_saliency después del relleno:",
      a_batch_saliency_padded.shape)  # Esperado: (8, 27, 768)
print("Dimensiones x_batch_embeddings después del relleno:",
      x_batch_embeddings_padded.shape)  # Esperado: (8, 27, 768)

print("a_batch_saliency_padded:", a_batch_saliency_padded.shape)
print("x_batch_embeddings_padded:", x_batch_embeddings_padded.shape)


# Transformar a un formato adecuado

# x_batch_embeddings_reduced = x_batch_embeddings_padded.mean(
#     axis=2)  # Promediar sobre la dimensión del embedding
# a_batch_saliency_reduced = a_batch_saliency_padded.mean(
#     axis=2)  # Promediar sobre la dimensión del embedding
# # x_batch_embeddings_reduced[:,:,0]
# x_batch_embeddings_reduced_long = x_batch_embeddings_reduced.long()
# # a_batch_saliency_reduced
# print(x_batch_embeddings_reduced_long.shape)

# # Convertir a tensores si es necesario
# x_batch_embeddings_padded_np = np.array(x_batch_embeddings_padded)
# a_batch_saliency_padded_np = np.array(a_batch_saliency_padded)
# Evaluar la métrica



scores_fidelity = metric_fidelity.evaluate_batch(
    model=roberta_model,
    x_batch=x_batch_embeddings_padded,
    y_batch=y_batch,
    a_batch=a_batch_saliency_padded,
    device='cpu'
)

# Imprimir los resultados de la evaluación
print("Scores de Estimación de Fidelidad:", scores_fidelity)

Dimensiones de x_batch: (8,)
Dimensiones de : (8,)
Dimensiones de a_batch_saliency: (8, 10, 2)
Dimensiones de embedding: torch.Size([8, 27, 768])
a_batch_saliency: [[[ 4.00000000e+00  7.90321762e-02]
  [ 2.00000000e+00  5.76310947e-02]
  [ 6.00000000e+00  5.76310947e-02]
  [ 1.00000000e+01  5.11234429e-02]
  [ 5.00000000e+00  5.11234429e-02]
  [ 0.00000000e+00 -2.79087333e-02]
  [ 9.00000000e+00 -2.79087333e-02]
  [ 1.00000000e+00 -2.14010815e-02]
  [ 8.00000000e+00  1.52457859e-02]
  [ 3.00000000e+00  6.50765176e-03]]

 [[ 9.00000000e+00  5.60289742e-02]
  [ 8.00000000e+00  3.95943188e-02]
  [ 3.00000000e+00 -1.64346554e-02]
  [ 2.00000000e+00  1.48726130e-02]
  [ 5.00000000e+00  1.48726130e-02]
  [ 7.00000000e+00  1.48726130e-02]
  [ 1.20000000e+01  1.48726130e-02]
  [ 1.50000000e+01  1.48726130e-02]
  [ 0.00000000e+00 -1.56204241e-03]
  [ 1.00000000e+00 -1.56204241e-03]]

 [[ 2.00000000e+00  4.77117669e-02]
  [ 4.00000000e+00  4.77117669e-02]
  [ 6.00000000e+00  4.77117669e-02]
  [ 

AttributeError: 'RobertaForSequenceClassification' object has no attribute 'shape_input'

In [ ]:
import quantus
import torch

# Supongamos que ya tienes las siguientes variables definidas:
# x_batch, y_batch, a_batch_saliency, embeddings

# Crear una instancia de la métrica Estimación de Fidelidad
metric_fidelity = quantus.FaithfulnessCorrelation()
print(x_batch)
# Obtener las dimensiones necesarias
batch_size = x_batch_reduced.shape[0]  # Número de batches
desired_length = x_batch_reduced.shape[1]  # Longitud deseada para x_batch

# Transformar a un formato adecuado
# Seleccionar el primer valor del embedding (por ejemplo, el primer token)
# Esto tendrá forma (batch_size, hidden_size)
x_batch_reduced = embeddings[:, 0, :]

# Asegúrate de que a_batch_saliency_reduced tenga la misma forma
# Esto tiene forma (3, 10)
a_batch_saliency_reduced = a_batch_saliency[:, :, 1]
# Convertir a_batch_saliency_reduced a un tensor si es un ndarray
if isinstance(a_batch_saliency_reduced, np.ndarray):
    a_batch_saliency_reduced = torch.tensor(a_batch_saliency_reduced)
# Crear un nuevo tensor lleno de ceros con la forma deseada
new_a_batch_saliency = torch.zeros((batch_size, desired_length))

# Copiar los valores de a_batch_saliency_reduced al nuevo tensor
new_a_batch_saliency[:, :a_batch_saliency_reduced.shape[1]
                     ] = a_batch_saliency_reduced
new_a_batch_saliency = np.array(new_a_batch_saliency)
# Verificar dimensiones después del relleno
print("Dimensiones transformadas:",
      new_a_batch_saliency.shape)  # Debería ser (3, 27)
print(x_batch_reduced)
print(new_a_batch_saliency)

# Debería ser (3, hidden_size)
print("Dimensiones de x_batch_reduced:", x_batch_reduced.shape)
# Debería ser (3, desired_length)
print("Dimensiones de new_a_batch_saliency:", new_a_batch_saliency.shape)

# Evaluar la métrica
scores_fidelity = metric_fidelity(
    model=roberta_model,
    x_batch=x_batch_reduced,
    y_batch=y_batch,
    a_batch=new_a_batch_saliency,
    device='cpu',
    subset_size=min(10, x_batch_reduced.shape[0])
)

# Imprimir los resultados de la evaluación
print("Scores de Estimación de Fidelidad:", scores_fidelity)

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from utils.utils import load_dataset, save_explanation
from lime.lime_text import LimeTextExplainer
from models.roberta_model import classify_comment, load_roberta_model_main
import torch
import quantus
from quantus.metrics.faithfulness.faithfulness_estimate import FaithfulnessEstimate

# Configuración del dispositivo
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Cargar el conjunto de datos
dataset = load_dataset(
    "models/relevance_classification/dataset/facebook_labeled.csv", sep=",")

# Cargar el modelo RoBERTa preentrenado
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta_model = AutoModelForSequenceClassification.from_pretrained(
    "modelo de ray mejorado(SHAP)/")
roberta_model.eval()  # Poner el modelo en modo de evaluación

# Configurar el explicador LIME
explainer_roberta = LimeTextExplainer(class_names=['not relevant', 'relevant'])

# Seleccionar una instancia del conjunto de datos para explicar
instance_roberta = dataset['Review'].iloc[0]

# Generar la explicación usando LIME
explanation_roberta = explainer_roberta.explain_instance(
    instance_roberta,
    classify_comment
)
print("Etiquetas disponibles:", explainer_roberta.available_labels())

# Extraer las características y las puntuaciones de relevancia del explicador LIME
a_batch_saliency = explanation_roberta.as_map(
)[explanation_roberta.top_labels[0]]

# Convertir a un formato adecuado para Quantus
# Aquí debes asegurarte de que a_batch_saliency esté en el formato correcto (ndarray)

# Convertir a_batch_saliency a un ndarray si es necesario
a_batch_saliency_array = np.array(
    [saliency[1] for saliency in a_batch_saliency])

# Crear un objeto de la métrica Estimación de Fidelidad
metric_fidelity = FaithfulnessEstimate()

# Preparar x_batch y y_batch para la evaluación (asegúrate de que estén en el formato correcto)
x_batch = [instance_roberta]  # Aquí deberías tener tus instancias como texto
# La etiqueta predicha por tu modelo
y_batch = [classify_comment(instance_roberta)]

# Evaluar la métrica de fidelidad
scores_fidelity = metric_fidelity(
    model=roberta_model,
    x_batch=x_batch,
    y_batch=y_batch,
    a_batch=a_batch_saliency_array,
    device=device,
)

# Imprimir los resultados de la evaluación de fidelidad
print("Scores de Estimación de Fidelidad:", scores_fidelity)